# Ensemble Learning

## Initial Imports

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [24]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Read the CSV and Perform Basic Data Cleaning

In [25]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [20]:
# Some of the data needs to be relabled and converted to an integer so it can be read by the scaler
#le = LabelEncoder()
#le.fit(df["home_ownership"])
#df["home_ownership"] = le.transform(df["home_ownership"])
#le.fit(df["verification_status"])
#df["verification_status"] = le.transform(df["verification_status"])
#df["issue_d"] = le.fit_transform(df["issue_d"])
#le.fit(df["pymnt_plan"])
#df["pymnt_plan"] = le.transform(df["pymnt_plan"])
#encoded_df = pd.get_dummies(df)



In [26]:
# Create our features
X = pd.get_dummies(df.drop(columns='loan_status'))

# Create our target
y = df['loan_status']

In [27]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [28]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [29]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y
)
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [30]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [31]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [32]:
# Scale the training and testing data
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [36]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, 
        random_state=1, 
        class_weight="balanced"
        )
rf_model = rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

In [37]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.6147088565776698

In [39]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, 
    index=["Actual High_Risk", "Actual Low_Risk"], 
    columns=["Predicted High_Risk", "Predicted Low_Risk"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,20,67
Actual Low_Risk,8,17110


In [40]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.71      0.23      1.00      0.35      0.48      0.21        87
   low_risk       1.00      1.00      0.23      1.00      0.48      0.25     17118

avg / total       0.99      1.00      0.23      0.99      0.48      0.25     17205



In [41]:
# List the features sorted in descending order by feature importance
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.07988895497634205, 'total_rec_prncp'),
 (0.06842428360058145, 'total_rec_int'),
 (0.062118546007423586, 'total_pymnt_inv'),
 (0.058562288801462956, 'last_pymnt_amnt'),
 (0.057688399907900874, 'total_pymnt'),
 (0.027584808094830954, 'int_rate'),
 (0.02596634995570926, 'issue_d_Jan-2019'),
 (0.020767484457315608, 'issue_d_Mar-2019'),
 (0.01930759387467497, 'installment'),
 (0.017791527433642314, 'out_prncp_inv'),
 (0.017203421600735392, 'out_prncp'),
 (0.017007869572941866, 'annual_inc'),
 (0.01676753937174969, 'dti'),
 (0.014879839362736102, 'avg_cur_bal'),
 (0.014758828832193777, 'max_bal_bc'),
 (0.014718127256938643, 'mths_since_recent_inq'),
 (0.014669255110425921, 'mo_sin_old_rev_tl_op'),
 (0.01440046239679605, 'bc_util'),
 (0.014087234695765253, 'il_util'),
 (0.013723992030873232, 'revol_bal'),
 (0.013700264752502857, 'tot_hi_cred_lim'),
 (0.013633281242959339, 'total_bc_limit'),
 (0.013472570996836946, 'mo_sin_old_il_acct'),
 (0.013414069617265696, 'bc_open_to_buy'),
 (0.01293

### Easy Ensemble Classifier

In [43]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=500, 
        random_state=1, 
        )

eec_model = eec_model.fit(X_train, y_train)
predictions = eec_model.predict(X_test)

In [44]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.9263620468069506

In [45]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, 
    index=["Actual High_Risk", "Actual Low_Risk"], 
    columns=["Predicted High_Risk", "Predicted Low_Risk"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,79,8
Actual Low_Risk,947,16171


In [46]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier had the best score.  92.6 vs 61.5

2. Which model had the best recall score?

    The Easy Ensemble Classifier had the best recall score.  Despite the low_risk having a score of 1, the Random Forest Classifier had a very low recall score for the high_risk data set.  The Easy Ensemble Classifier had both recall values above 0.9.

3. Which model had the best geometric mean score?

    The Easy Ensemble Classiffier had the best geometric mean score at 0.93 vs 0.48.

4. What are the top three features?

    The top three features were 'total_rec_prncp', 'total_rec_int', and 'total_pymnt_inv'